In [1]:
import pandas as pd
import difflib as dlb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('movies.csv')
df.head(5)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [3]:
df.shape

(4803, 24)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [5]:
#selecting relevant columns
relevant_features = df[['genres', 'keywords', 'tagline', 'title', 'vote_average', 'cast']]
relevant_features

,genres,keywords,tagline,title,vote_average,cast
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Avatar,7.2,Sam Worthington Zoe Saldana Sigourney Weaver S...
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,Johnny Depp Orlando Bloom Keira Knightley Stel...
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Spectre,6.3,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,The Dark Knight Rises,7.6,Christian Bale Michael Caine Gary Oldman Anne ...
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",John Carter,6.1,Taylor Kitsch Lynn Collins Samantha Morton Wil...
...,...,...,...,...,...,...
4798,Action Crime Thriller,united states\u2013mexico barrier legs arms pa...,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,Carlos Gallardo Jaime de Hoyos Peter Marquardt...
4799,Comedy Romance,NaN,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...
4800,Comedy Drama Romance TV Movie,date love at first sight narration investigati...,NaN,"Signed, Sealed, Delivered",7.0,Eric Mabius Kristin Booth Crystal Lowe Geoff G...
4801,NaN,NaN,A New Yorker in Shanghai,Shanghai Calling,5.7,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...


In [6]:
relevant_features.isnull().sum()

genres           28
keywords        412
tagline         844
title             0
vote_average      0
cast             43
dtype: int64

In [7]:
#replacing missing values
for col in relevant_features.columns:
    relevant_features[col] = relevant_features[col].fillna('')

In [8]:
relevant_features.isnull().sum()

genres          0
keywords        0
tagline         0
title           0
vote_average    0
cast            0
dtype: int64

In [9]:
# combining all features into a single feature
description = relevant_features['genres']+' '+relevant_features['keywords']+' '+relevant_features['tagline']+' '+relevant_features['cast']
description.head()

0    Action Adventure Fantasy Science Fiction cultu...
1    Adventure Fantasy Action ocean drug abuse exot...
2    Action Adventure Crime spy based on novel secr...
3    Action Crime Drama Thriller dc comics crime fi...
4    Action Adventure Science Fiction based on nove...
dtype: object

In [10]:
tfidf_vect = TfidfVectorizer()
vectors = tfidf_vect.fit_transform(description)
print(vectors)

  (0, 11944)	0.2179541049534567
  (0, 9358)	0.16220625760325083
  (0, 8035)	0.24268136037924426
  (0, 13376)	0.1679762975847774
  (0, 15268)	0.20043369094168031
  (0, 12876)	0.2080378726062569
  (0, 12211)	0.21994287729015155
  (0, 15836)	0.20401594060427147
  (0, 15585)	0.23881752360753422
  (0, 12235)	0.16150825899870827
  (0, 10522)	0.2748571666270283
  (0, 10234)	0.09140575746628334
  (0, 15576)	0.12950556500906077
  (0, 13980)	0.0716738996919006
  (0, 4570)	0.24268136037924426
  (0, 13069)	0.2160790405184415
  (0, 2965)	0.2521186835379356
  (0, 15189)	0.12675984407319427
  (0, 13166)	0.3430524228349123
  (0, 5381)	0.16633572090148666
  (0, 2817)	0.22432333413714167
  (0, 3392)	0.2160790405184415
  (0, 5022)	0.1046865481338198
  (0, 12451)	0.1046865481338198
  (0, 4878)	0.11220584657020999
  :	:
  (4801, 15819)	0.31669946803986226
  (4801, 4465)	0.2711909206056166
  (4801, 385)	0.20162062799136785
  (4801, 6373)	0.31669946803986226
  (4801, 10669)	0.23809938363906946
  (4801, 1533)

In [11]:
similarity_score = cosine_similarity(vectors)
print(similarity_score)

[[1.         0.07580156 0.01525023 ... 0.         0.         0.        ]
 [0.07580156 1.         0.03515988 ... 0.03787357 0.         0.        ]
 [0.01525023 0.03515988 1.         ... 0.         0.03266233 0.        ]
 ...
 [0.         0.03787357 0.         ... 1.         0.         0.0354845 ]
 [0.         0.         0.03266233 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.0354845  0.         1.        ]]


In [12]:
# Save the vectorizer and similarity scores
with open('tfidf_vectorizer.pkl', 'wb') as vec_file:
    pickle.dump(tfidf_vect, vec_file)

# Save the similarity scores
with open('similarity_scores.pkl', 'wb') as sim_file:
    pickle.dump(similarity_score, sim_file)

In [14]:
name = input('Enter a movie name: ')
titles = df['title'].tolist()
get_close_match = dlb.get_close_matches(name, titles)
closest_match = get_close_match[0]
index_of_movie = df[df.title == closest_match]['index'].values[0]
sim_score = list(enumerate(similarity_score[index_of_movie]))
similar_movies = sorted(sim_score, key=lambda x: x[1], reverse=True)

# Collect details for the top 10 recommendations to determine column widths
recommendations = []
i = 1
for movie in similar_movies:
    Index = movie[0]
    movie_details = df[df.index == Index][['title', 'genres', 'vote_average']].iloc[0]
    title = movie_details['title']
    genre = movie_details['genres']
    rating = f"{movie_details['vote_average']:.1f}"
    
    if i <= 10:
        recommendations.append((i, title, rating, genre))
        i += 1

# Create a DataFrame to hold the recommendations
recommendations_df = pd.DataFrame(recommendations, columns=["No", "Title", "Rating", "Genre"])

# Display the recommendations
print("Similar Movie Recommendations for you:\n")
print(recommendations_df)

Enter a movie name:  spiderman


Similar Movie Recommendations for you:

   No                      Title Rating                            Genre
0   1                 Spider-Man    6.8                   Fantasy Action
1   2               Spider-Man 3    5.9         Fantasy Action Adventure
2   3               Spider-Man 2    6.7         Action Adventure Fantasy
3   4                  Clerks II    6.9                           Comedy
4   5                 Seabiscuit    6.7                    Drama History
5   6               The Notebook    7.7                    Romance Drama
6   7              The Ice Storm    6.9                            Drama
7   8            Horrible Bosses    6.5                     Comedy Crime
8   9  The Count of Monte Cristo    7.3  Action Adventure Drama Thriller
9  10            In Good Company    5.9             Comedy Drama Romance
